In [36]:
import sys
import os

sys.path.append(os.path.abspath('../'))  # añade ecg_project_full al path


In [37]:
import numpy as np, os, matplotlib.pyplot as plt
from scripts.preprocess import preprocess_signal, segment_beats
from scripts.train_cvae import train, CVAE
from scripts.extract_latents import extract_errors_and_latents
from classifiers.train_classifier import train_classifier
from utils.metrics import compute_roc, compute_pr, optimal_threshold

In [ ]:
# En la primera celda de tu pipeline_lead_II.ipynb:
raw_dir   = r"C:/Users/anapt/Repositorios/TP-final_ML/TP-final_ML\data"
beats_dir = r"C:/Users/anapt/Repositorios/TP-final_ML/TP-final_ML\data/processed/beats"
full_dir  = r"C:/Users/anapt/Repositorios/TP-final_ML/TP-final_ML\data/processed/full"

os.makedirs(beats_dir, exist_ok=True)
os.makedirs(full_dir,  exist_ok=True)

valid_files = [f for f in os.listdir(raw_dir) if f.endswith('.npy')]
print("Archivos raw encontrados:", valid_files)

for fname in valid_files:
    data = np.load(os.path.join(raw_dir, fname))
    p = preprocess_signal(data)
    b = segment_beats(p)
    # Sólo guardamos si realmente hay beats
    if b.size:
        np.save(os.path.join(beats_dir, fname.replace('.npy','_beats.npy')), b)
        np.save(os.path.join(full_dir, fname), p)

print("Preprocessing done")
print("Archivos en beats_dir:", os.listdir(beats_dir))


Preprocessing done


In [39]:
import os

print("Archivos en beats_dir:", os.listdir(beats_dir))


Archivos en beats_dir: []


In [40]:
valid_files = [f for f in os.listdir(beats_dir) if f.endswith('.npy')]
print("Archivos .npy válidos:", valid_files)


Archivos .npy válidos: []


In [41]:
# Train CVAE
import tensorflow as tf
beats=[]
# for f in os.listdir(beats): beats.append(np.load(os.path.join(beats,f)))
# beats=np.concatenate(beats,0)
# dataset=tf.data.Dataset.from_tensor_slices(beats[...,None]).batch(32)
# model=train(dataset,epochs=50)

for f in os.listdir(beats_dir):
    if f.endswith('.npy'):
        beats.append(np.load(os.path.join(beats_dir, f)))

if beats:
    beats = np.concatenate(beats, axis=0)
    dataset = tf.data.Dataset.from_tensor_slices(beats[..., None]).batch(32)
    model = train(dataset, epochs=50)
else:
    print("No se encontraron datos de beats para entrenar.")


No se encontraron datos de beats para entrenar.


In [42]:
# Extract
extract_errors_and_latents('cvae_beats_weights.h5', beats, 'errors.npy', 'z_mean.npy')
errors, latents = np.load('errors.npy'), np.load('z_mean.npy')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'cvae_beats_weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Classifier
probs, y_test = train_classifier('errors.npy','z_mean.npy')

In [ ]:
# Metrics and plots
fpr,tpr,thr,roc=compute_roc(y_test,probs)
prec,rec,pr_thr,pr=compute_pr(y_test,probs)
best=optimal_threshold(fpr,tpr,thr)
print(f'AUC-ROC={roc:.3f}, AUC-PR={pr:.3f}, best_thr={best:.3f}')
plt.figure();plt.plot(fpr,tpr,label=f'AUC={roc:.3f}');plt.plot([0,1],[0,1],'k--');plt.legend();plt.show()
plt.figure();plt.plot(rec,prec,label=f'AUC-PR={pr:.3f}');plt.legend();plt.show()